# Structured Output

Now I will show you how to leverage **structured output**.

It allows you to get structured data like JSON from the AI model, which is particularly useful when you integrate the results into an application.

With Magentic, we do this using Pydantic models to specify the shape of the data we expect and force the LLM to follow it.

Then we get Pydantic models as output.

Filter out some Pydantic warnings

In [ ]:
import warnings
warnings.filterwarnings("ignore")

Necessary imports

In [ ]:
from dotenv import load_dotenv
from magentic import chatprompt, AssistantMessage, SystemMessage, UserMessage, prompt
from magentic.chat_model.litellm_chat_model import LitellmChatModel
from pydantic import BaseModel
from typing import Literal
from datetime import datetime
from pprint import pprint


load_dotenv()

# Create Some Unstructured Data

I have gathered data from 2 reviews on TrustPilot for ChatGPT.

`review_1` is positive while `review_2` is negative.

In [ ]:
review_1 = """
Rated 5 out of 5 stars
Nov 16, 2023
ChatGPT is a lovely tool that provides…
ChatGPT is a lovely tool that provides quite good answers. With the wolfram alpha plugin it does calculations very good also.

Date of experience: November 15, 2023
"""

review_2 = """
Rated 1 out of 5 stars
Sep 14, 2023
ChatGPT looping itself.
It is repeating itself. I tell it I want to do something without doing something else, It gives me something that doesn't work. I tell it that, then it gives me something that includes what I don't want. It even does it in the same response!
"If you have an object that you want to write to a JSON file without it being stringified, you should first convert it to a JSON string using JSON.stringify and then write it to the file."

Date of experience: September 14, 2023
"""

# Define A Pydantic Model For Reviews

In [ ]:
class Review(BaseModel):
    sentiment: Literal["positive", "negative", "neutral"]
    grade: int
    summary: str
    date: datetime

# Structured Output From `@prompt`

In [ ]:
@prompt("Extract a Review from: {review}", model=LitellmChatModel("gpt-3.5-turbo"))
def extract(review: str) -> Review: ...

print("Parsing review 1:")
res = extract(review_1)
pprint(res.model_dump())

print()

print("Parsing review 2:")
res = extract(review_2)
pprint(res.model_dump())

# Structured Output From `@chatpromt`

In [7]:
@chatprompt(
    SystemMessage("You are an expert to extract structured data from reviews."),
    UserMessage("Extract a Review from this review: {review}?"),
    model=LitellmChatModel("gpt-3.5-turbo"),
)
def extract_chat(review: str) -> Review: ...


print("Parsing review 1:")
res = extract_chat(review_1)
pprint(res.model_dump())

print()

print("Parsing review 2:")
res = extract_chat(review_2)
pprint(res.model_dump())

Parsing review 1:
{'date': datetime.datetime(2023, 11, 16, 0, 0),
 'grade': 5,
 'sentiment': 'positive',
 'summary': 'ChatGPT is a lovely tool'}

Parsing review 2:
{'date': datetime.datetime(2023, 9, 14, 0, 0),
 'grade': 1,
 'sentiment': 'negative',
 'summary': 'ChatGPT repeating and providing incorrect information'}
